In [90]:
from pathlib import Path
from functools import reduce

from rasa.shared.nlu.training_data.formats.rasa_yaml import RasaYAMLReader
from rasa.shared.nlu.training_data.training_data import TrainingData

In [91]:
data = RasaYAMLReader().reads(Path('.data/nlu_2intents-0.yml').read_text('utf-8'))

### Keep only contact_service and contact_doctor

In [92]:
INTENT_TO_KEEP = ['contact_service', 'contact_doctor']

# Filter examples by intent

filtered_examples = list(filter(lambda x: x.data['intent'] in INTENT_TO_KEEP, data.training_examples))
print(f'{len(filtered_examples)}/{len(data.training_examples)} examples kept')

# Replace currents training_examples

data.training_examples.clear()
data.training_examples.extend(filtered_examples)

76/101 examples kept


### Remove "target" role in entities

In [93]:
# For each entity in each example -> remove 'role' key
for example in data.training_examples:
    for entity in example.data['entities']:
        entity.pop('role', '')

### Add service and doctor lookup tables from training data

In [96]:
# Load train nlu file
train_data = RasaYAMLReader().reads(Path('data/2intents-0/nlu.yml').read_text('utf-8'))

# Set lookup_tables
data.lookup_tables = train_data.lookup_tables

### Save result as test file

In [97]:
data.persist_nlu('tests/nlu_2intents-0.yml')